In [4]:
import tf_transformations
import numpy as np
from geometry_msgs.msg import Transform

In [16]:
def message_from_transform(T):
    '''
    http://docs.ros.org/kinetic/api/geometry_msgs/html/msg/Transform.html

    geometry_msgs/Vector3 translation
    geometry_msgs/Quaternion rotation
    '''
    msg = Transform()
    q = tf_transformations.quaternion_from_matrix(T)
    translation = tf_transformations.translation_from_matrix(T)
    msg.translation.x = translation[0]
    msg.translation.y = translation[1]
    msg.translation.z = translation[2]
    msg.rotation.x = q[0]
    msg.rotation.y = q[1]
    msg.rotation.z = q[2]
    msg.rotation.w = q[3]
    return msg

def get_rotation_matrix_from_vector(vector):
    # Normalize the input vector (new x-axis)
    vector = np.array(vector)
    x_axis = vector / np.linalg.norm(vector)
    
    # Create an arbitrary vector that is not parallel to the input vector
    arbitrary_vec = np.array([0, 0, 1]) if np.allclose(x_axis, [1, 0, 0]) else np.array([1, 0, 0])
    
    # Compute the new z-axis as the cross product of the new x-axis and the arbitrary vector
    z_axis = np.cross(x_axis, arbitrary_vec)
    z_axis /= np.linalg.norm(z_axis)  # Normalize the z-axis
    
    # Compute the new y-axis as the cross product of the new z-axis and the new x-axis
    y_axis = np.cross(z_axis, x_axis)
    
    # Return the rotation matrix (new basis)
    rotation_matrix = np.column_stack((x_axis, y_axis, z_axis))
    
    # Create the 4x4 transformation matrix
    transformation_matrix = np.eye(4)  # Start with an identity matrix
    transformation_matrix[:3, :3] = rotation_matrix  # Set the upper left 3x3 to the rotation matrix
    
    return transformation_matrix

def inverse_transformation_matrix(matrix):
    # Extract the rotation part (upper-left 3x3) and translation part (last column, first 3 elements)
    rotation_matrix = matrix[:3, :3]
    translation_vector = matrix[:3, 3]
    
    # Invert the rotation matrix (since it is orthogonal, the inverse is the transpose)
    rotation_matrix_inv = np.transpose(rotation_matrix)
    
    # Invert the translation: -R_inv * T
    translation_inv = -np.dot(rotation_matrix_inv, translation_vector)
    
    # Construct the inverse transformation matrix
    inverse_matrix = np.eye(4)  # Start with an identity matrix
    inverse_matrix[:3, :3] = rotation_matrix_inv  # Set the rotation part
    inverse_matrix[:3, 3] = translation_inv  # Set the translation part
    
    return inverse_matrix


In [6]:
T1 = tf_transformations.concatenate_matrices(
    	    tf_transformations.quaternion_matrix(tf_transformations.quaternion_from_euler(0.79, 0.0, 0.79)),
    	    tf_transformations.translation_matrix((0.0, 1.0, 1.0))
        )
object_transform = message_from_transform(T1)

In [8]:
T2 = tf_transformations.concatenate_matrices(
        tf_transformations.quaternion_matrix(tf_transformations.quaternion_about_axis(1.5, (0.0, 0.0, 1.0))),
        tf_transformations.translation_matrix((0.0,-1.0,0.0))
    )
robot_transform = message_from_transform(T2)

In [23]:
camera_translation_matrix = tf_transformations.translation_matrix((0.0,0.1,0.1))

camera_transform_in_base_frame = tf_transformations.concatenate_matrices(
    T2,
    camera_translation_matrix
)
base_transform_in_camera_frame = inverse_transformation_matrix(camera_transform_in_base_frame)

array([[ 7.07372017e-02,  9.97494987e-01,  0.00000000e+00,
         1.38777878e-17],
       [-9.97494987e-01,  7.07372017e-02,  0.00000000e+00,
         9.00000000e-01],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
        -1.00000000e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [41]:
object_origin_in_base_frame = T1[:,3]
object_origin_in_camera_frame = np.matmul(base_transform_in_camera_frame,object_origin_in_base_frame)

In [43]:
R3 = get_rotation_matrix_from_vector(object_origin_in_camera_frame[:3])
T3 = tf_transformations.concatenate_matrices(
            camera_translation_matrix,
            R3   
        )
T3

array([[-0.0026679 ,  0.99999644,  0.        ,  0.        ],
       [ 0.56291406,  0.0015018 ,  0.82651407,  0.1       ],
       [ 0.82651113,  0.00220506, -0.56291606,  0.1       ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])